In [25]:
import csv
import pandas as pd
import nltk
import time
import numpy as np
import string, re

nltk.download('words')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay, RocCurveDisplay
from sklearn.model_selection import GridSearchCV


from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB

[nltk_data] Downloading package words to /Users/andrewlin/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/andrewlin/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# Data Preprocessing

In [ ]:
# Import Song Data

song_data = pd.read_csv('./data/lyrics-data.csv')
english_songs = song_data.loc[song_data['language'] == 'en']
english_songs.rename(columns={'ALink':'Link'}, inplace=True)
english_songs.head()

In [ ]:
# Import Artist Data

artist_data = pd.read_csv('./data/artists-data.csv')
artist_data = artist_data.drop_duplicates(subset='Link', keep='first')
artist_data.head()

In [ ]:
# Merge Dataframes

combined_data = pd.merge(english_songs, artist_data, on='Link')
combined_data = combined_data[["SName", "Artist", "Lyric", "Genres"]]
combined_data = combined_data[combined_data.Lyric.notnull()]
combined_data = combined_data[combined_data.Genres.notnull()]
combined_data

In [ ]:
artists = combined_data.Artist.unique()
genres = combined_data.Genres.unique()
labels = ["Blues", "Country", "Hip Hop", "Jazz", "Pop", "Rock"]

In [ ]:
# Randomely select 1000 songs from each genre

rows = []

for label in labels:
    genre_data = combined_data[combined_data.Genres == label]
    genre_data = genre_data.sample(n=1000, replace=False)
    genre_data.Genres = label
    rows.append(genre_data)

combined_genre_data = pd.concat(rows).reset_index().drop("index", axis=1)
combined_genre_data

In [ ]:
combined_genre_data.to_csv("combined_genre_data.csv")

## Reload the data

In [4]:
combined_genre_data = pd.read_csv("combined_genre_data.csv")
combined_genre_data.head()

,Unnamed: 0,SName,Artist,Lyric,Genres
0,0,Country Boy,Muddy Waters,Don't say I don't love you\nBecause I stays ou...,Blues
1,1,Don't Wait 'til Tomorrow,Gary Clark Jr.,I'm so tired of fighting\nI'm so tired of bein...,Blues
2,2,Frankie,Mississippi John Hurt,"(spoken introduction:\n""Frankie and Albert"", t...",Blues
3,3,Fear Of Falling,The Allman Brothers Band,"by G. Allman, J. Townsend & T. Heding\nThere w...",Blues
4,4,I've Always Been Lonely,BB King,I don't have to know your name\nUnless you rea...,Blues


In [10]:
lyrics_list = list(combined_genre_data.Lyric)
pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
lemmatizer = WordNetLemmatizer()
# stemmer = SnowballStemmer("english")
english = list(set(nltk.corpus.words.words()))

def clean_docs_lemma(lyrics_list):
    cleaned = []
    for lyric in lyrics_list:
        clean_lyric = nltk.regexp_tokenize(lyric, pattern)
        lyric_lower = [i.lower() for i in clean_lyric]
        stop_words = stopwords.words('english')
        stopwords_list = stop_words + list(string.punctuation)
        stopwords_list += ["''", '""', '...', '``']
        lyrics_tokens_stopped = [w for w in lyric_lower if not w in stopwords_list]
        lyric_lemmas = [lemmatizer.lemmatize(word) for word in lyrics_tokens_stopped]
        c = " ".join(lyric_lemmas)
        cleaned.append(c)
    return cleaned

In [11]:
lemmed_lyrics = clean_docs_lemma(lyrics_list)
len(lemmed_lyrics)

6000

In [12]:
X = lemmed_lyrics
y = combined_genre_data.Genres
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25) 

In [23]:
pipe_NB = Pipeline([('count_vectorizer', CountVectorizer()), 
                     ('tfidf_vectorizer', TfidfTransformer()),
                     ('clf', MultinomialNB())
                    ])

pipe_RF = Pipeline([('count_vectorizer', CountVectorizer()), 
                     ('tfidf_vectorizer', TfidfTransformer()),
                     ('clf', RandomForestClassifier())
                    ])

pipe_KNN = Pipeline([('count_vectorizer', CountVectorizer()), 
                     ('tfidf_vectorizer', TfidfTransformer()),
                     ('clf', KNeighborsClassifier())
                    ])

# List of pipelines, List of pipeline names
pipelines = [pipe_NB, pipe_RF, pipe_KNN]
pipeline_names = ['Multinomail NB', 'Random Forest', "KNN"]

# Loop to fit each of the three pipelines
for pipe in pipelines:
    print(pipe)
    pipe.fit(X_train, y_train)
    
# Compare accuracies
lemmed_basic_scores = []
for index, val in enumerate(pipelines):
    tup = (pipeline_names[index], val.score(X_test, y_test))
    lemmed_basic_scores.append(tup)
    print('%s pipeline test accuracy: %.3f' % (pipeline_names[index], val.score(X_test, y_test)))

Pipeline(steps=[('count_vectorizer', CountVectorizer()),
                ('tfidf_vectorizer', TfidfTransformer()),
                ('clf', MultinomialNB())])
Pipeline(steps=[('count_vectorizer', CountVectorizer()),
                ('tfidf_vectorizer', TfidfTransformer()),
                ('clf', RandomForestClassifier())])
Pipeline(steps=[('count_vectorizer', CountVectorizer()),
                ('tfidf_vectorizer', TfidfTransformer()),
                ('clf', KNeighborsClassifier())])
Multinomail NB pipeline test accuracy: 0.561
Random Forest pipeline test accuracy: 0.576
KNN pipeline test accuracy: 0.370


# Random Forest Classifier

In [26]:
GS_pipe_RF = Pipeline([('count_vectorizer', CountVectorizer()), 
                     ('tfidf_vectorizer', TfidfTransformer()),
                     ('clf', RandomForestClassifier())
                    ])

rf_param_grid = dict(clf__n_estimators = [10, 30, 100],
                  clf__criterion = ['gini', 'entropy'],
                  clf__max_depth = [2, 6, 10], 
                  clf__min_samples_split = [5, 10],
                  clf__min_samples_leaf = [3, 6])


gs_RF = GridSearchCV(estimator=GS_pipe_RF,
            param_grid=rf_param_grid,
            scoring='accuracy',
            cv=3)

gs_RF.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('count_vectorizer', CountVectorizer()),
                                       ('tfidf_vectorizer', TfidfTransformer()),
                                       ('clf', RandomForestClassifier())]),
             param_grid={'clf__criterion': ['gini', 'entropy'],
                         'clf__max_depth': [2, 6, 10],
                         'clf__min_samples_leaf': [3, 6],
                         'clf__min_samples_split': [5, 10],
                         'clf__n_estimators': [10, 30, 100]},
             scoring='accuracy')

In [27]:
rf_gs_testing_score = gs_RF.score(X_test, y_test)

print("Mean Testing Score: {:.4}%".format(rf_gs_testing_score * 100))
print("Best Parameter Combination Found During Grid Search: \n{}".format(gs_RF.best_params_))

Mean Testing Score: 51.73%
Best Parameter Combination Found During Grid Search: 
{'clf__criterion': 'gini', 'clf__max_depth': 10, 'clf__min_samples_leaf': 6, 'clf__min_samples_split': 10, 'clf__n_estimators': 100}


# Naive Bayes

In [28]:
GS_pipe_NB = Pipeline([('count_vectorizer', CountVectorizer()), 
                     ('tfidf_vectorizer', TfidfTransformer()),
                     ('clf', MultinomialNB())
                    ])


sorted(GS_pipe_NB.get_params().keys())

nb_param_grid = dict(clf__alpha = [.01, .05, .1, .2, .4, 1])

gs_NB = GridSearchCV(estimator=GS_pipe_NB,
            param_grid=nb_param_grid,
            scoring='accuracy',
            cv=3)

gs_NB.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('count_vectorizer', CountVectorizer()),
                                       ('tfidf_vectorizer', TfidfTransformer()),
                                       ('clf', MultinomialNB())]),
             param_grid={'clf__alpha': [0.01, 0.05, 0.1, 0.2, 0.4, 1]},
             scoring='accuracy')

In [29]:
nb_gs_testing_score = gs_NB.score(X_test, y_test)

print("Mean Testing Score: {:.4}%".format(nb_gs_testing_score * 100))
print("Best Parameter Combination Found During Grid Search: \n{}".format(gs_NB.best_params_))

Mean Testing Score: 61.27%
Best Parameter Combination Found During Grid Search: 
{'clf__alpha': 0.05}


# KNN Classifier

In [30]:
GS_pipe_KNN = Pipeline([('count_vectorizer', CountVectorizer()), 
                     ('tfidf_vectorizer', TfidfTransformer()),
                     ('clf', KNeighborsClassifier())
                    ])

knn_param_grid = dict(clf__n_neighbors = [100, 150, 200],
                     clf__p = [1, 2])

gs_KNN = GridSearchCV(estimator=GS_pipe_KNN,
            param_grid=knn_param_grid,
            scoring='accuracy',
            cv=3)

gs_KNN.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('count_vectorizer', CountVectorizer()),
                                       ('tfidf_vectorizer', TfidfTransformer()),
                                       ('clf', KNeighborsClassifier())]),
             param_grid={'clf__n_neighbors': [100, 150, 200], 'clf__p': [1, 2]},
             scoring='accuracy')

In [31]:
knn_gs_testing_score = gs_KNN.score(X_test, y_test)

print("Mean Testing Score: {:.4}%".format(knn_gs_testing_score * 100))
print("Best Parameter Combination Found During Grid Search: \n{}".format(gs_KNN.best_params_))

Mean Testing Score: 49.2%
Best Parameter Combination Found During Grid Search: 
{'clf__n_neighbors': 200, 'clf__p': 2}
